# Instituto Tecnologico de Costa Rica
# Ingenieria en Computacion
# Inteligencia artificial
---
## Tarea Programada 4
## Profesora : Maria Auxiliadora Mora
---
## Estudiantes:
## Kenneth Corrales Lizano 2017151342
## Fabian Vives Castro 2018319257
---


In [ ]:
# Bibliotecas
